# I want to develop my skills in data manipulation and Pytorch 

In [27]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [7]:
!pip uninstall pytorch

In [ ]:
!pip install torch

In [10]:
import torch
import torch.nn as nn

## Importing Data
 Below I searched for a suitable dataset to train and test a model on, I found a very good dataset on heart pointers and heart attack
 in binary of 1 and 0.
 It's a really clean dataset and so will massively help with just focusing on PyTorch skills for now.

In [16]:
df = pd.read_csv('archive/heart.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trtbps    303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalachh  303 non-null    int64  
 8   exng      303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slp       303 non-null    int64  
 11  caa       303 non-null    int64  
 12  thall     303 non-null    int64  
 13  output    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB


Just want to do some exploratory data analysis for now to map out the dataset a bit. I am just going to plot age in a frequency chart.
I will also plot age and blood pressure...

In [26]:
age = np.array(df[['age']])
